In [ ]:
import os
import cv2 
import numpy as np
import imutils
import random
import time
import sys
import json

from PIL import ImageFont, ImageDraw, Image  

DATA_DIRECTORY = "C:/Users/Sander/Desktop/data"
TEST_OUTPUT = "C:/Users/Sander/Desktop/"

In [ ]:
# Adds text to a desktop shortcut image (below the icon)
def addShortcutText(image, text, font, icon_within_border, icon_text_max_height, icon_image_max_height):
    # Convert the image to RGB (OpenCV uses BGR)  
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA)  

    # Pass the image to PIL  
    image_pil = Image.fromarray(image_rgb)  
    draw = ImageDraw.Draw(image_pil)  

    # use a truetype font  
    w, _ = draw.textsize(text, font=font)
    _, h = draw.textsize("h", font=font)

    # Get text coordinates
    textX = int((image.shape[1] - w) / 2)
    textY = int((icon_text_max_height - h) / 2) + icon_within_border
    
    # Draw the text
    draw.text((textX, textY + icon_image_max_height), text, font=font, fill=(255, 255, 255, 255))
    
    # Get back the image to OpenCV  
    image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGBA2BGRA)
    
    return image

In [ ]:
# Adds text to a given background image
def createTextField(image, text, font, color, offset=0):
    # Additional space to left and right of text
    left_border = 0
    right_border = 15
    
    # Make sure background image is large enough in terms of width
    while image.shape[1] < 500:
        image = np.hstack((image, image))
        
    # Convert the image to RGB (OpenCV uses BGR)  
    textImage_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  

    # Pass the image to PIL  
    textImage_pil = Image.fromarray(textImage_rgb)  
    draw = ImageDraw.Draw(textImage_pil)  

    # use a truetype font  
    w, _ = draw.textsize(text, font=font)
    _, h = draw.textsize("h", font=font)

    # Get text coordinates
    textX = left_border
    textY = int((image.shape[0] - h) / 2) + offset
    
    # Draw the text
    draw.text((textX, textY), text, font=font, fill=color)

    # Get back the image to OpenCV  
    textImage = cv2.cvtColor(np.array(textImage_pil), cv2.COLOR_RGB2BGR)
    
    # Fit image to text size (remove remaining background)
    textImage = textImage[0:textImage.shape[1], 0:w+left_border+right_border]
    
    return textImage

In [ ]:
# Overlays a transparent image onto another transparent image
def insertTransparentImage(image, overlay, x1, x2, y1, y2):
    img = image.copy()
    
    alpha_overlay = overlay[:, :, 3] / 255.0
    alpha_image = 1.0 - alpha_overlay

    # Intersect by alpha-value
    for c in range(0, 3):
        img[y1:y2, x1:x2, c] = (alpha_overlay * overlay[:, :, c] + alpha_image * img[y1:y2, x1:x2, c])
        
    return img

In [ ]:
# Adds a border effect to a given image
def addWindowBorderEffect(window):
    # Get border size
    size = border_effect_size
    
    # Make image larger such that border fits
    image = np.zeros((window.shape[0]+size*2, window.shape[1]+size*2, 4))
    
    # Draw border line-by-line
    for i in range(0, size):
        # Get transparency of current line
        transparency = 255 / (2 ** (size-i))
        
        # Get border color
        color = (120, 120, 120, transparency)
        
        # Draw border line
        image[i:i+1, i:image.shape[1]-i] = color
        image[i:image.shape[0]-i, image.shape[1]-i-1:image.shape[1]-i] = color
        image[image.shape[0]-i-1:image.shape[0]-i, i:image.shape[1]-i] = color
        image[i:image.shape[0]-i, i:i+1] = color
    
    # Add window to the drawn border
    image[size:size+window.shape[0], size:size+window.shape[1]] = window
    
    return image

In [ ]:
# Add necessary paths
path_to_additional_data = DATA_DIRECTORY + "/mac_files/data/"
path_to_screenshots = DATA_DIRECTORY + "/mac_files/screens/"

path_to_icons = DATA_DIRECTORY + "/icons/"
path_to_wallpapers = DATA_DIRECTORY + "/mac_files/wallpapers/"

path_to_top_taskbar_elements = DATA_DIRECTORY + "/mac_files/taskbar_top/"
path_to_bottom_taskbar_elements = DATA_DIRECTORY + "/mac_files/taskbar_bottom/"

path_to_window_elements = DATA_DIRECTORY + "/mac_files/window/"
path_to_websites = DATA_DIRECTORY + "/websites/"

In [ ]:
# Load all required images

# Left part of topbar
taskbar_top_left_original = cv2.imread(os.path.join(path_to_additional_data, 'top_start.png'))
# Middle part of topbar
taskbar_top_middle = cv2.imread(os.path.join(path_to_additional_data, 'top_middle.png'))
# Right part of topbar
taskbar_top_right_original = cv2.imread(os.path.join(path_to_additional_data, 'top_end.png'))

# Left part of bottom taskbar
taskbar_bottom_left_original = cv2.imread(os.path.join(path_to_additional_data, 'left_end.png'), cv2.IMREAD_UNCHANGED)
# Horizontal separator icon of bottom taskbar
taskbar_bottom_separator_original = cv2.imread(os.path.join(path_to_additional_data, 'separator.png'))
taskbar_bottom_separator_original = cv2.cvtColor(taskbar_bottom_separator_original, cv2.COLOR_RGB2RGBA)
# Right part of bottom taskbar
taskbar_bottom_right_original = cv2.imread(os.path.join(path_to_additional_data, 'right_end.png'), cv2.IMREAD_UNCHANGED)
# Open task dot effect of bottom taskbar
taskbar_bottom_dot = cv2.imread(os.path.join(path_to_additional_data, 'task_dot.png'), cv2.IMREAD_UNCHANGED)

# Left part of window head
window_left = cv2.imread(os.path.join(path_to_window_elements, 'window_start.png'), cv2.IMREAD_UNCHANGED)
# Middle part of window head
window_middle = cv2.imread(os.path.join(path_to_window_elements, 'window_middle.png'))
window_middle = cv2.cvtColor(window_middle, cv2.COLOR_RGB2RGBA)
# Right part of window head
window_right = cv2.imread(os.path.join(path_to_window_elements, 'window_end.png'), cv2.IMREAD_UNCHANGED)
# Optinal extra elements of window head (left side)
window_left_optional1 = cv2.imread(os.path.join(path_to_window_elements, 'window_optinal1_left.png'))
window_left_optional1 = cv2.cvtColor(window_left_optional1, cv2.COLOR_RGB2RGBA)
window_left_optional2 = cv2.imread(os.path.join(path_to_window_elements, 'window_optinal2_left.png'))
window_left_optional2 = cv2.cvtColor(window_left_optional2, cv2.COLOR_RGB2RGBA)
# Optinal extra elements of window head (right side)
window_right_optional = cv2.imread(os.path.join(path_to_window_elements, 'window_optinal1_right.png'))
window_right_optional = cv2.cvtColor(window_right_optional, cv2.COLOR_RGB2RGBA)

# Left part of Safari browser head
safari_left = cv2.imread(os.path.join(path_to_window_elements, 'safari_left.png'), cv2.IMREAD_UNCHANGED)
# Middle part of Safari browser head
safari_middle = cv2.imread(os.path.join(path_to_window_elements, 'safari_middle.png'))
safari_middle = cv2.cvtColor(safari_middle, cv2.COLOR_RGB2RGBA)
# Right part of Safari browser head
safari_right = cv2.imread(os.path.join(path_to_window_elements, 'safari_right.png'), cv2.IMREAD_UNCHANGED)

# Path to fonts
path_to_font_normal = os.path.join(path_to_additional_data, 'LucidaGrande.ttf')
path_to_font_bold = os.path.join(path_to_additional_data, 'LucidaGrandeBold.ttf')

# Mouse icon
mouse_icon = cv2.imread(os.path.join(path_to_additional_data, 'mouse.png'), cv2.IMREAD_UNCHANGED)

# Screenshot size
wallpaper_width = 1920
wallpaper_height = 1080

# Width of border effect for application windows
border_effect_size = 5

# Scale mouse to screenshot size
scale = wallpaper_width / 110
mouse_icon = imutils.resize(mouse_icon, width=int(scale))

# Scale topbar to screenshot size
scale = wallpaper_height / 47
taskbar_top_left_original = imutils.resize(taskbar_top_left_original, height=int(scale))
taskbar_top_middle = imutils.resize(taskbar_top_middle, height=int(scale))
taskbar_top_right_original = imutils.resize(taskbar_top_right_original, height=int(scale))

# Scale winodw head to screenshot size
scale = wallpaper_height / 25
window_left = imutils.resize(window_left, height=int(scale))
window_middle = imutils.resize(window_middle, height=int(scale))
window_right = imutils.resize(window_right, height=int(scale))
window_left_optional1 = imutils.resize(window_left_optional1, height=int(scale))
window_left_optional2 = imutils.resize(window_left_optional2, height=int(scale))
window_right_optional = imutils.resize(window_right_optional, height=int(scale))

# Scale Safari browser head to screenshot size
scale = wallpaper_height / 30
safari_left = imutils.resize(safari_left, height=int(scale))
safari_middle = imutils.resize(safari_middle, height=int(scale))
safari_right = imutils.resize(safari_right, height=int(scale))

In [ ]:
# Create an empty desktop image
def createDesktop():
    # Copy lbottom taskbar elements
    taskbar_bottom_left = taskbar_bottom_left_original.copy()
    taskbar_bottom_separator = taskbar_bottom_separator_original.copy()
    taskbar_bottom_right = taskbar_bottom_right_original.copy()
    
    # Copy topbar elements
    taskbar_top_left = taskbar_top_left_original.copy()
    taskbar_top_right = taskbar_top_right_original.copy()
    
    # Select a random wallpaper
    if len(os.listdir(path_to_wallpapers)) == 0:
        print('Did not find any wallpapers in ' + path_to_wallpapers)
    wallpaper_file = random.choice(os.listdir(path_to_wallpapers))
    wallpaper = cv2.imread(os.path.join(path_to_wallpapers, wallpaper_file))
    
    # Only scales to width!
    # wallpaper = imutils.resize(wallpaper, width=wallpaper_width)
    wallpaper = cv2.resize(wallpaper, (wallpaper_width, wallpaper_height))
    
    # ==> Bottom Taskbar:
    # Select a random number of icon elements for bottom taskbar
    number_of_bottom_elements = random.randint(8, len(os.listdir(path_to_bottom_taskbar_elements)))

    # Scale bottom taskbar according to number of icons in it
    scale = wallpaper.shape[0] / (7 + (number_of_bottom_elements / 3))
    taskbar_bottom_left = imutils.resize(taskbar_bottom_left, height=int(scale))
    taskbar_bottom_separator = imutils.resize(taskbar_bottom_separator, height=int(scale))
    taskbar_bottom_right = imutils.resize(taskbar_bottom_right, height=int(scale))
    
    # Get the random icons
    samples = random.sample(os.listdir(path_to_bottom_taskbar_elements), k=number_of_bottom_elements)

    # Get a random number of separators between the icons
    number_of_separators = random.randint(0, int(number_of_bottom_elements / 5))
    separators = random.sample(range(1, int(number_of_bottom_elements/2)), k=number_of_separators)
    
    # There are at least two elements between a separator
    separators = sorted([i * 2 for i in separators])

    # Add icons to bar
    for i in range(0, len(samples)):
        # Load and resize icon
        icon = cv2.imread(os.path.join(path_to_bottom_taskbar_elements, samples[i]))
        icon = cv2.cvtColor(icon, cv2.COLOR_RGB2RGBA)
        icon = imutils.resize(icon, height=taskbar_bottom_left.shape[0])

        # Decide if we add a "selected" dot to the icon
        coinflip = random.randint(0, 4)
        if coinflip == 0:
            # Add dot to icon
            dot_overlay = imutils.resize(taskbar_bottom_dot, width=icon.shape[1])
            icon = insertTransparentImage(icon, dot_overlay, 0, dot_overlay.shape[1], icon.shape[0] - dot_overlay.shape[0], icon.shape[0])
        
        # Append icon to taskbar
        taskbar_bottom_left = np.hstack((taskbar_bottom_left, icon))

        # Check if we need to add a separator
        if i in separators:
            # Append separator
            taskbar_bottom_left = np.hstack((taskbar_bottom_left, taskbar_bottom_separator))
        
        # Abort if taskbar exceeds screenshot size
        if wallpaper.shape[1] - taskbar_bottom_left.shape[1] - taskbar_bottom_right.shape[1] - taskbar_bottom_separator.shape[1] - icon.shape[1] < 80:
            break

    # Add right end of bottom taskbar
    taskbar_bottom_left = np.hstack((taskbar_bottom_left, taskbar_bottom_right))

    # Place taskbar at bottom center
    x1 = int((wallpaper.shape[1] - taskbar_bottom_left.shape[1]) / 2)
    x2 = x1 + taskbar_bottom_left.shape[1]

    y1 = wallpaper.shape[0] - taskbar_bottom_left.shape[0]
    y2 = y1 + taskbar_bottom_left.shape[0]

    # Draw taskbar on wallpaper
    wallpaper = insertTransparentImage(wallpaper, taskbar_bottom_left, x1, x2, y1, y2)
    
    # ==> Top Taskbar (left):
    # Vocabulary for topbar menu items
    words = ["Lorem", "Ipsum", "Dolor", "Consetetur", "Sadipscing", "Elitr", "Nonumy", "Eirmod", "Tempor", "Invidunt",
             "Labore", "Dolore", "Magna", "Aliquyam", "Erat", "Voluptua", "Vero", "Accusam", "Dolores", "Rebum"]

    # Get a random title name
    text = " ".join(random.sample(words, k=random.randint(1,2)))

    # Get background for text
    textImage = taskbar_top_middle.copy()

    # Get fonts
    fontsize = int(textImage.shape[0] / 1.5)
    font_normal = ImageFont.truetype(path_to_font_normal, fontsize)  
    font_bold = ImageFont.truetype(path_to_font_bold, fontsize)

    # Draw text on background
    textImage = createTextField(textImage, text, font_bold, (0, 0, 0, 255))
    
    # Append title text to left side of topbar
    taskbar_top_left = np.hstack((taskbar_top_left, textImage))

    # Add a random number of menu items
    number_of_elements = random.randint(2, 7)
    # Get random menu words
    fields = random.sample(words, k=number_of_elements)

    # Append menu words to left side of topbar
    for i in range(0, len(fields)):
        textImage = taskbar_top_middle.copy()
        textImage = createTextField(textImage, fields[i], font_normal, (0, 0, 0, 255))

        # Append to topbar
        taskbar_top_left = np.hstack((taskbar_top_left, textImage))

    # Draw left part of topbar on wallpaper
    x1 = 0
    x2 = x1 + taskbar_top_left.shape[1]
    
    y1 = 0
    y2 = y1 + taskbar_top_left.shape[0]

    wallpaper[y1:y2, x1:x2] = taskbar_top_left

    
    # ==> Top Taskbar (right):

    # Get a random number of icons
    number_of_elements = random.randint(2, len(os.listdir(path_to_top_taskbar_elements)))
    icons = random.sample(os.listdir(path_to_top_taskbar_elements), k=number_of_elements)

    # Append icons to right part of topbar
    for i in range(0, len(icons)):
        icon = cv2.imread(os.path.join(path_to_top_taskbar_elements, icons[i]))
        icon = imutils.resize(icon, height=taskbar_top_right.shape[0])

        # Append to topbar (right)
        taskbar_top_right = np.hstack((icon, taskbar_top_right))

    # Draw right topbar on wallpaper
    x1 = wallpaper.shape[1] - taskbar_top_right.shape[1]
    x2 = x1 + taskbar_top_right.shape[1]
    
    y1 = 0
    y2 = y1 + taskbar_top_right.shape[0]

    wallpaper[y1:y2, x1:x2] = taskbar_top_right

    # ==> Top Taskbar (middle):

    # Fill the middle part between left and right topbar with the filler image
    # Compute remaining filler size
    fillwidth = wallpaper.shape[1] - taskbar_top_right.shape[1] - taskbar_top_left.shape[1]
    
    # Get y-coordinate
    y_offset= 0

    # If there is still space we need to fill, then... (filler image is not infinite, so we may need to fill up multiple times)
    if fillwidth > 0:
        i = 0
        while i < fillwidth:
            # Get x-coordinate
            x_offset = taskbar_top_left.shape[1] + i
            
            # Cut image if whole image would be larger than fillwidth
            width = min(taskbar_top_middle.shape[1], fillwidth - i)

            # Add filler image
            wallpaper[y_offset:y_offset+taskbar_top_middle.shape[0], x_offset:x_offset+width] = taskbar_top_middle[:, 0:width]

            # Update fillwidth
            i += taskbar_top_middle.shape[1]
        
    return wallpaper, taskbar_bottom_left.shape, wallpaper_file

In [ ]:
# Show empty desktop image
desktop, bottom_shape, _ = createDesktop()

cv2.imshow("Empty Desktop Image", desktop)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite(TEST_OUTPUT + str(int(time.time()))+ ".png", desktop)

In [ ]:
# Create a random window head for a given width
def generateWindowHead(final_width):
    # Copy window head elements
    start = window_left.copy()
    end = window_right.copy()

    # Randomly decide if we add optional element 1 on left end
    coinflip = random.randint(0, 1)

    if coinflip == 0:
        start = np.hstack((start, window_left_optional1))
    
    # Randomly decide if we add optional element 2 on left end
    coinflip = random.randint(0, 1)

    if coinflip == 0:
        start = np.hstack((start, window_left_optional2))
    
    # Randomly decide if we add optional element 1 on right end
    coinflip = random.randint(0, 1)

    if coinflip == 0:
        end = np.hstack((window_right_optional, end))  
    
    # Application name vocabulary
    words = ["Lorem", "Ipsum", "Dolor", "Consetetur", "Sadipscing", "Elitr", "Nonumy", "Eirmod", "Tempor", "Invidunt",
             "Labore", "Dolore", "Magna", "Aliquyam", "Erat", "Voluptua", "Vero", "Accusam", "Dolores", "Rebum"]

    # Create random application name
    text = " ".join(random.sample(words, k=random.randint(1,3)))

    # Get background for text title
    textImage = window_middle.copy()

    # Get font
    fontsize = int(textImage.shape[0] / 2.5)
    font = ImageFont.truetype(path_to_font_normal, fontsize)  

    # Draw title text on background
    textImage = createTextField(textImage, text, font, (0, 0, 0, 255))
    textImage = cv2.cvtColor(textImage, cv2.COLOR_RGB2RGBA)

    # Fill remaining width (left side) with filler image (title is in the middle)
    fillwidth = int((final_width - textImage.shape[1]) / 2) - start.shape[1]
    y_offset = 0

    if fillwidth > 0:
        # There is still space to fill, so fill
        i = 0
        while i < fillwidth:
            # Cut image if whole image would be larger than fillwidth
            width = min(window_middle.shape[1], fillwidth - i)
            
            # Append filler image
            start = np.hstack((start, window_middle[:, 0:width]))
            
            # Update remaining fillwidth
            i += window_middle.shape[1]
    else:
        # Application title is already too large. Cut such that at least left and right end still fit to it
        textImage = textImage[:, 0:(final_width - start.shape[1] - end.shape[1])]
        
    
    # Add text title to the middle
    start = np.hstack((start, textImage))
    
    # Fill remaining width (right side) with filler image (title is in the middle)
    fillwidth = final_width - start.shape[1] - end.shape[1]
    y_offset = 0

    if fillwidth > 0:
        # There is still space to fill, so fill
        i = 0
        while i < fillwidth:
            # Cut image if whole image would be larger than fillwidth
            width = min(window_middle.shape[1], fillwidth - i)
            
            # Append filler image
            start = np.hstack((start, window_middle[:, 0:width]))
            
            # Update remaining fillwidth
            i += window_middle.shape[1]
    
    # Add right end of window head (close window, etc...)
    start = np.hstack((start, end))
    
    return start

# Create a random browser head for a given width
def generateBrowserHead(final_width):
    start = safari_left.copy()
    
    # Get a random URL
    words = ["lorem", "ipsum", "dolor", "sit", "amet", "consetetur", "sadipscing", "elitr", "sed", 
             "diam", "nonumy", "eirmod", "tempor", "invidunt", "ut", "labore", "et", "dolore", "magna", 
             "aliquyam", "erat", "sed", "diam", "voluptua", "at", "vero", "eos", "et", "accusam", "et", 
             "justo", "duo", "dolores", "rebum"]

    # Seperator characters which connect the random URL words
    separators = ["/", ".", "-", "_", "=", "+", ""]
    
    # Create random URL
    samples = random.sample(words, k=random.randint(3,15))
    text = ""
    
    # Add random seperators between words
    for s in samples:
        sep = random.choice(separators)
        text += s
        text += sep
    
    # Get background for URL
    textImage = safari_middle.copy()
    
    # Get URL font
    fontsize = int(textImage.shape[0] / 3.5)
    font = ImageFont.truetype(path_to_font_normal, fontsize) 

    # Add URL text to background
    textImage = createTextField(textImage, text, font, (0, 0, 0, 255), offset=-2)
    
    # Fill remaining width (left side) with filler image (URL is in the middle)
    fillwidth = int((final_width - textImage.shape[1]) / 2) - start.shape[1]
    y_offset = 0

    if fillwidth > 0:
        # There is still space to fill, so fill
        i = 0
        while i < fillwidth:
            # Cut image if whole image would be larger than fillwidth
            width = min(safari_middle.shape[1], fillwidth - i)
            
            # Append filler image
            start = np.hstack((start, safari_middle[:, 0:width]))
            
            # Update remaining fillwidth
            i += safari_middle.shape[1]
    else:
        # Browser head is already too large. Cut such that at least left and right end still fit to it
        textImage = textImage[:, 0:(final_width - safari_right.shape[1] - safari_left.shape[1])]
    
    # Append title to middle
    textImage = cv2.cvtColor(textImage, cv2.COLOR_RGB2RGBA)
    start = np.hstack((start, textImage))
    
    # Fill remaining width (right side) with filler image (URL is in the middle)
    fillwidth = final_width - start.shape[1] - safari_right.shape[1]
    y_offset = 0

    if fillwidth > 0:
        # There is still space to fill, so fill
        i = 0
        while i < fillwidth:
            # Cut image if whole image would be larger than fillwidth
            width = min(safari_middle.shape[1], fillwidth - i)
            
            # Append filler image
            start = np.hstack((start, safari_middle[:, 0:width]))
            
            # Update remaining fillwidth
            i += safari_middle.shape[1]
    
    # Add right end of browser head
    start = np.hstack((start, safari_right))
    
    return start

In [ ]:
# Add a random browser window to the desktop screenshot
def addBrowerWindow(screen, website_name, fullscreen, bottom_shape):
# Captures whether we take the large or small version of a website
    version = "large"
    
    # If application is not shown in fullscreen, randomly decide if we go for small or large version
    if not fullscreen:
        coinflip = random.randint(0, 1)
        
        if coinflip == 0:
            version = "small"
    
    # Load website screenshot
    website_path = os.path.join(path_to_websites, version, website_name)
    website = cv2.imread(website_path)
    
    if website is None:
        print(website_path)
    
    # Disable fullscreen mode for webpages which would be rescaled too large
    if website.shape[0] < screen.shape[0] / 2 or website.shape[1] < screen.shape[1] / 2:
        #print(website_path, " is too small for fullscreen mode!")
        fullscreen = False
    
    # Take into account the bottom taskbar
    additional_offset = 0
    if not fullscreen:
        additional_offset = bottom_shape[0]
    
    # Resize website such that it is not lager than the screenshot
    website = imutils.resize(website, width=min(screen.shape[1] - border_effect_size * 2, website.shape[1]))
    website = imutils.resize(website, height=min(screen.shape[0] - taskbar_top_left_original.shape[0] - safari_right.shape[0] - additional_offset - border_effect_size * 2, website.shape[0]))   
        
    if fullscreen:
        # Resize webpage to fullscreen
        website = cv2.resize(website, (screen.shape[1], screen.shape[0] - taskbar_top_left_original.shape[0] - safari_right.shape[0]))
    else:
        # Randomly scale the webpage, since it's not in fullscreen
        scale = random.uniform(0.7, 1.0)
        website = imutils.resize(website, width=int(website.shape[1] * scale))
               
    # Generate a random application and browser head for webpage
    head = generateWindowHead(website.shape[1])
    browserHead = generateBrowserHead(website.shape[1])
    
    website = cv2.cvtColor(website, cv2.COLOR_RGB2RGBA)
    website = np.vstack((browserHead, website))
    
    # Add window effect to webpage (if not in fullscreen)
    if not fullscreen:
        website = addWindowBorderEffect(website)
    
    # Choose random position for application
    x1 = random.randint(0, screen.shape[1] - website.shape[1])
    x2 = x1+website.shape[1]
    
    y1 = random.randint(taskbar_top_left_original.shape[0], screen.shape[0] - website.shape[0] - additional_offset)
    y2 = y1+website.shape[0]

    # Add webpage to screenshot
    screen = insertTransparentImage(screen, website, x1, x2, y1, y2)
    
    data = {"name": website_path, "fullscreen": fullscreen, "x1": x1, "x2": x2, "y1": y1, "y2": y2}
    
    return screen, data

# Add a random application window to the desktop screenshot
def addApplicationWindow(screen, app_path, fullscreen, bottom_shape):
    # Load application screenshot
    app = cv2.imread(app_path)
    
    # Disable fullscreen mode for applications which would be rescaled too large
    if app.shape[0] < screen.shape[0] / 1.5 or app.shape[1] < screen.shape[1] / 1.5:
        #print(website_path, " is too small for fullscreen mode!")
        fullscreen = False
        
    # Check if we need to take into account the bottom taskbar (if not in fullscreen)
    additional_offset = 0
    if not fullscreen:
        additional_offset = window_right.shape[0] + bottom_shape[0]
    
    # Resize application such that it is not lager than the screenshot
    app = imutils.resize(app, width=min(screen.shape[1] - border_effect_size * 2, app.shape[1]))
    app = imutils.resize(app, height=min(screen.shape[0] - taskbar_top_left_original.shape[0] - additional_offset - border_effect_size * 2, app.shape[0]))
           
    if fullscreen:
        # Resize application to fullscreen
        app = cv2.resize(app, (screen.shape[1], screen.shape[0] - taskbar_top_left_original.shape[0]))
    else:
        # Randomly scale the application, since it's not in fullscreen
        scale = random.uniform(0.7, 1.0)
        app = imutils.resize(app, width=int(app.shape[1] * scale))
            
    app = cv2.cvtColor(app, cv2.COLOR_RGB2RGBA)
      
    # Generate a random application head if necessary
    if not fullscreen:
        #head = generateWindowHead(app.shape[1])
        #app = np.vstack((head, app))
        pass

    # Add window effect to application (if not in fullscreen)
    if not fullscreen:
        app = addWindowBorderEffect(app)

    # Choose random position for application
    x1 = random.randint(0, screen.shape[1] - app.shape[1])
    x2 = x1+app.shape[1]
    
    y1 = random.randint(taskbar_top_left_original.shape[0], screen.shape[0] - app.shape[0] - additional_offset)
    y2 = y1+app.shape[0]

    # Add application to screenshot
    screen = insertTransparentImage(screen, app, x1, x2, y1, y2)
    
    data = {"name": app_path, "fullscreen": fullscreen, "x1": x1, "x2": x2, "y1": y1, "y2": y2}
    
    return screen, data

In [ ]:
# Create a random desktop shortcut
def createShortcut(icon_name, icon_size):
    # Vocabulary for shortcut text
    words = ["lorem", "ipsum", "dolor", "sit", "amet", "consetetur", "sadipscing", "elitr", "sed", 
         "diam", "nonumy", "eirmod", "tempor", "invidunt", "ut", "labore", "et", "dolore", "magna", 
         "aliquyam", "erat", "sed", "diam", "voluptua", "at", "vero", "eos", "et", "accusam", "et", 
         "justo", "duo", "dolores", "rebum"]

    # Padding of the shortcut
    icon_within_border = int(icon_size * 0.26)

    # Height of the icon
    icon_image_max_height = int(icon_size * 0.8) - icon_within_border
    
    # Height of the shortcut text
    icon_text_max_height = icon_size - icon_image_max_height - icon_within_border

    # Get the shortcut font
    fontsize = int(icon_within_border / 1.9)
    font_normal = ImageFont.truetype(path_to_font_normal, fontsize)  
    font_bold = ImageFont.truetype(path_to_font_bold, fontsize)
    
    # Load the shortcut icon
    icon = cv2.imread(os.path.join(path_to_icons, icon_name), cv2.IMREAD_UNCHANGED)

    # Max possible width of icon
    max_width = min(icon_size - 2*icon_within_border, icon.shape[1])
    icon = imutils.resize(icon, width=max_width)

    # Max possible height of icon
    max_height = min(icon_image_max_height, icon.shape[0])
    icon = imutils.resize(icon, height=max_height)

    # Center icon 
    x = int((icon_size - icon.shape[1]) / 2)
    y = icon_within_border + int((icon_image_max_height - icon.shape[0]) / 2)

    # Add the icon
    image = np.zeros((icon_size, icon_size, 4), np.uint8)
    image[y:y+icon.shape[0], x:x+icon.shape[1]] = icon

    # Generate the random text
    text = " ".join(random.choices(words, k=random.randint(1,3)))
    
    # Add text
    image = addShortcutText(image, text, font_bold, icon_within_border, icon_text_max_height, icon_image_max_height)
    
    return image

In [ ]:
# Fills the desktop screenshot with content
def fillDesktop(screen, bottom_shape):
    # ==> Desktop Icons:
    
    coinflip = random.randint(0, 2)
    
    if coinflip == 0:
        # Select random number of shortcuts
        number_of_desktop_icons = random.randint(6, 30)

        # Load icons for shortcuts
        icon_files = os.listdir(path_to_icons)
        icons = random.choices(icon_files, k=number_of_desktop_icons)

        # Define shortcut size
        icon_size = int(screen.shape[0] / 10)

        # Define additional (horizontal) margin between shortcuts
        icon_outside_border = int(icon_size * 0.05)

        for i in range(0, len(icons)):
            # Create shortcut
            image = createShortcut(icons[i], icon_size)

            # Choose a random position
            x1 = random.randint(10, screen.shape[1] - image.shape[1] - 10)
            x2 = x1 + image.shape[1]

            y1 = random.randint(10 + taskbar_top_left_original.shape[0], screen.shape[0] - bottom_shape[0] - image.shape[0] - 10)
            y2 = y1 + image.shape[0]

            # Draw shortcut on screen
            screen = insertTransparentImage(screen, image, x1, x2, y1, y2)
    elif coinflip == 1:
        pass
    else:
        # Define shortcut size
        icon_size = int(screen.shape[0] / 10)

        # Define additional (horizontal) margin between shortcuts
        icon_outside_border = int(icon_size * 0.05)

        # Compute grid size for shortcuts
        row_size = int((screen.shape[0] - taskbar_top_left_original.shape[0] - bottom_shape[0]) / icon_size)
        col_size = int((screen.shape[1] - icon_outside_border) / (icon_size + icon_outside_border))

        # Enlarge horizontal margin between shortcuts such that there is no potential large gap at end
        horizontal_rest_outside_border = screen.shape[1] - (col_size * icon_size + (col_size+1) * icon_outside_border)
        horizontal_outside_border = int(horizontal_rest_outside_border / (col_size + 1)) + icon_outside_border

        # Enlarge vertical margin between shortcuts such that there is no potential large gap at end
        vertical_rest_outside_border = screen.shape[0] - (row_size * icon_size + (row_size+1) * icon_outside_border)- bottom_shape[0] - 10
        vertical_outside_border = int(vertical_rest_outside_border / (row_size + 1)) + icon_outside_border

        # Chance that there is a block of shortcuts at the border of the screen
        # The larger the number, the less likely
        border_chance = random.randint(8, 14)

        # Chance that there is a block of shortcuts within the screen
        normal_chance = random.randint(border_chance*2, border_chance*4)

        # Grid matrix which hold where a shortcut is placed (1 shortcut, 0 no shortcut)
        icon_matrix = np.zeros((row_size, col_size))

        # Traverse the border of the grid (counter clockwise)
        steps = [[1, 0], [0, 1], [-1, 0], [0, -1]]

        x = 0
        y = 0
        i = 0

        while True:
            # Check if we hit a corner, if so, change the traverse direction
            if y + steps[i][0] < 0 or y + steps[i][0] == icon_matrix.shape[0]:
                i += 1
            elif x + steps[i][1] < 0 or x + steps[i][1] == icon_matrix.shape[1]:
                i += 1

            # We did one whole turn around the border, stop
            if i == 4:
                break

            # Get next grid field
            x += steps[i][1]
            y += steps[i][0]

            # If there already is an shortcut, continue
            if icon_matrix[y, x] == 1:
                continue

            # Decide if there is a shortcut
            coinflip = random.randint(0, border_chance)

            if coinflip == 0:
                # We place a shortcut. Now decide how many surrounding shortcuts there are

                # Vertical direction
                if steps[i][0] == 1 or (steps[i][0] == 0 and y == 0):
                    # We are at left border (going down) or at the top border (going left)
                    y1 = y
                    y2 = y + random.randint(0, min(2, icon_matrix.shape[0] - y - 1)) + 1
                else:
                    y1 = y - random.randint(0, min(2, y))
                    y2 = y + 1

                # Horizontal direction
                if steps[i][1] == 1 or (steps[i][1] == 0 and x == 0):
                    # We are at bottom border (going right) or at the left border (going down)
                    x1 = x
                    x2 = x + random.randint(0, min(4, icon_matrix.shape[1] - x - 1)) + 1
                else:
                    x1 = x - random.randint(0, min(4, x))
                    x2 = x + 1

                # Fill with 1s (i.e., add block of shortcuts)
                icon_matrix[y1:y2, x1:x2] = 1

        # Select a random number of desktop icons which will be added to the left side of screen (one.after.the.other)
        number_of_icons = random.randint(0, 12)
        for i in range(0, number_of_icons):
            row = int(i / row_size)
            col = i % row_size

            icon_matrix[col, row] = 1

        # Decide for shortcut blocks within the screenshot (not border)
        for y in range(0, icon_matrix.shape[0]):
            for x in range(0, icon_matrix.shape[1]):
                if icon_matrix[y, x] == 1:
                    continue

                # Randomly decide if we add a shortcut
                coinflip = random.randint(0, normal_chance)

                if coinflip == 0:
                    # Decide for size of shortcut block
                    height = y + random.randint(1, min(2, icon_matrix.shape[0] - y))
                    width = x + random.randint(1, min(3, icon_matrix.shape[1] - x))

                    icon_matrix[y:height, x:width] = 1

        #print(icon_matrix)

        # Draw shortcuts to the screenshot
        icon_files = os.listdir(path_to_icons)

        for i in range(0, row_size):
            for j in range(0, col_size):
                if icon_matrix[i, j] == 1:
                    # Select random icon
                    icon_name = random.choice(icon_files)
                    image = createShortcut(icon_name, icon_size)

                    # Compute position
                    x1 = j * icon_size + (j+1) * horizontal_outside_border
                    x2 = x1 + image.shape[1]

                    y1 = i * icon_size + (i+1) * vertical_outside_border
                    y2 = y1 + image.shape[0]

                    # Draw on screenshot
                    screen = insertTransparentImage(screen, image, x1, x2, y1, y2)
    
    # ==> Application Windows:
    applications = []
    
    # Decide for a random number of application windows
    number_of_applications = random.randint(0, 5)
    
    # Load all application windows from dataset
    for img in os.listdir(path_to_screenshots):
        applications.append(os.path.join(path_to_screenshots, img))
    
    # Sample random screenshots
    app_paths = []
    if len(applications) >= number_of_applications:
        # Only do this if we have provided enough application screenshots
        app_paths = random.sample(applications, k=number_of_applications)
    else:
        print(path_to_screenshots + ' does not contain enough application screenshots! Want ' + str(number_of_applications) + ' but found ' + str(len(applications)))

    # Load webpage screenshots from dataset
    websites = os.listdir(os.path.join(path_to_websites, "large"))
    
    # Sample random webpages
    website_names = []
    if len(websites) >= number_of_applications:
        # Only do this if we have provided enough website screenshots
        website_names = random.sample(websites, k=number_of_applications)
    else:
        print(os.path.join(path_to_websites, "large") + ' does not contain enough website screenshots! Want ' + str(number_of_applications) + ' but found ' + str(len(websites)))
    
    data = []
    
    # Draw windows
    for i in range(0, number_of_applications):
        coinflip = random.randint(0, 4)
    
        # Decide if fullscreen or not
        if coinflip == 0:
            fullscreen = True
        else:
            fullscreen = False
        
        # Decide if webpage or application 
        coinflip = random.randint(0, 2)
        
        if coinflip == 0 and len(app_paths) > i:
            screen, d = addApplicationWindow(screen, app_paths[i], fullscreen, bottom_shape)
        elif len(website_names) > i:
            screen, d = addBrowerWindow(screen, website_names[i], fullscreen, bottom_shape)
        else:
            break
            
        data.append(d)
    
    # ==> Mouse Icon:
    
    # Get a random position for mouse icon
    x1 = random.randint(0, screen.shape[1] - mouse_icon.shape[1])
    x2 = x1 + mouse_icon.shape[1]
    
    y1 = random.randint(0, screen.shape[0] - mouse_icon.shape[0])
    y2 = y1 + mouse_icon.shape[0]

    # Draw mouse icon
    #screen = insertTransparentImage(screen, mouse_icon, x1, x2, y1, y2)
    
    return screen, data

In [ ]:
# Randomly fill screenshot
screen = desktop.copy()
screen, _ = fillDesktop(screen, bottom_shape)

cv2.imshow("Desktop Image", screen)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite(TEST_OUTPUT + str(int(time.time())) + ".png", screen)

In [ ]:
# Prints the progress in the console
def print_progress(count, total):
    pct_complete = float(count) / total
    msg = "\r- Progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()

In [ ]:
# Output path of synthesized screenshots
output_path = "C:/Users/Sander/Desktop/Dataset/Out1/Mac/"

if not os.path.exists(output_path):
    os.mkdir(output_path)
    
# Number of synthesized screenshots
number_of_images = 100

# Create random screenshots
for i in range(0, number_of_images):
    print_progress(i+1, number_of_images)
    image, bottom_shape, wallpaperID = createDesktop()
    image, data = fillDesktop(image, bottom_shape)
    cv2.imwrite(output_path + str(i+1) + ".png", image)
    
    with open(output_path + str(i+1) + ".json", 'w') as json_file:
        json.dump({"background" : wallpaperID, "applications" : data}, json_file)

print()
print("Terminated")